In [2]:
import pandas as pd
import numpy as np
from corsikaio import CorsikaFile
import re
import struct
import os

In [3]:
filename = 'pDAT163010'
metadata_filename = f'{filename}.dbase'
file_path = f'data/p30/{filename}' 
metadata_file_path = f'data/p30/{metadata_filename}' 

save_path = f'csv_otput/{filename}_output' 

In [4]:
os.makedirs(save_path, exist_ok=True) 

In [21]:
def parse_metadata(file_path):
    metadata = {}

    with open(file_path, 'r') as f:
        for line in f:
            line = line.strip() 

            match = re.search(r'#howmanyshowers#\s*(\d+)', line)
            if match:
                metadata['howmanyshowers'] = int(match.group(1))

            match = re.search(r'#energy_prim#\s*([\d\.eE\+\-]+)', line)
            if match:
                metadata['energy_prim'] = float(match.group(1))

            match = re.search(r'#theta_prim#\s*([\d\.eE\+\-]+)', line)
            if match:
                metadata['theta_prim'] = float(match.group(1))

            match = re.search(r'#phi_prim#\s*([\d\.eE\+\-]+)', line)
            if match:
                metadata['phi_prim'] = float(match.group(1))

            match = re.search(r'#dsn_events#\s*(\S+)', line)
            if match:
                metadata['dsn_events'] = match.group(1)

    return metadata

In [22]:
PID_MAP = {
    1: "gamma", 2: "e-", 3: "e+", 4: "ν_e", 5: "μ-", 6: "μ+",
    7: "π⁰", 8: "π+", 9: "π-", 13: "n", 14: "p", 75: "Fe",
    25: "Λ⁰", 35: "Σ⁺", 45: "Ξ⁰", 55: "Ω⁻", 65: "anti-proton",
}

def pid_name(pid_raw):
    """Преобразует 'сырой' PID в читаемое имя"""
    base_pid = pid_raw // 1000
    return PID_MAP.get(base_pid, f"unknown({base_pid})")


In [23]:
def save_event_data(event_idx, particles, header, save_path):
    theta_prim = header.get('theta_prim', 0)
    phi_prim = header.get('phi_prim', 0)
    energy_prim = header.get('energy_prim', 0)

    
    event_data = []

    for p in particles:
        raw_pid = int(p[0])
        base_pid = raw_pid // 1000
        name = pid_name(raw_pid)
        
        px, py, pz = p[1], p[2], p[3]
        x, y, t = p[4] / 100, p[5] / 100, p[6]
        energy = np.sqrt(px**2 + py**2 + pz**2)
        
        event_data.append({
            'event_id': event_idx,
            'particle_name': name,
            'pid': base_pid,
            'x': x,
            'y': y,
            't': t,
            'energy': energy,
            'theta_prim': theta_prim,
            'phi_prim': phi_prim,
            'energy_prim': energy_prim,
        })
    
    df_event = pd.DataFrame(event_data)
    event_file = os.path.join(save_path, f"event_{event_idx}.csv")
    df_event.to_csv(event_file, index=False)
    print(f"Сохранены данные события {event_idx} в файл: {event_file}")


In [24]:
with CorsikaFile(file_path) as f:
    for event_idx, event in enumerate(f):
        header = event.header
        particles = event.data
        
        metadata = parse_metadata(metadata_file_path) 
        if isinstance(header, np.void):
            header = {field: header[field] for field in header.dtype.names}
        
        header.update(metadata) 
        save_event_data(event_idx, particles, header, save_path)

Сохранены данные события 0 в файл: csv_otput/pDAT163010_output\event_0.csv
Сохранены данные события 1 в файл: csv_otput/pDAT163010_output\event_1.csv
Сохранены данные события 2 в файл: csv_otput/pDAT163010_output\event_2.csv
Сохранены данные события 3 в файл: csv_otput/pDAT163010_output\event_3.csv
Сохранены данные события 4 в файл: csv_otput/pDAT163010_output\event_4.csv


In [5]:
params_dict: dict[int, [float]] = {}

def parse_corsika_showers(file_path):
    """
    Читает бинарный файл CORSIKA и группирует данные по шалам:
      - Заголовок 'EVTH' задаёт начало события.
      - Заголовок 'EVTE' содержит параметры NeNKG и sNKG.
    При нахождении блока RUNE обработка завершается.
    """
    current_shower = None 
    shower_count = 0     

    with open(file_path, 'rb') as f:
        while True:
            marker_bytes = f.read(4)
            if len(marker_bytes) < 4:
                break

            record_size = struct.unpack('i', marker_bytes)[0]
            if record_size <= 0:
                print("Некорректный размер записи:", record_size)
                break

            block_bytes = f.read(record_size)
            if len(block_bytes) != record_size:
                print("Ошибка чтения: получено", len(block_bytes), "байт, ожидалось", record_size)
                break

            end_marker = f.read(4)
            if len(end_marker) < 4:
                print("Неверное завершение записи.")
                break

            num_floats = record_size // 4
            block = np.frombuffer(block_bytes, dtype=np.float32, count=num_floats)

            num_subblocks = 21
            subblock_size = 273

            for j in range(num_subblocks):
                start = j * subblock_size
                end = start + subblock_size
                if end > len(block):
                    break

                subblock = block[start:end]
                header_bytes = subblock[0].tobytes()
                try:
                    header_str = header_bytes.decode('ascii')
                except UnicodeDecodeError:
                    header_str = ""

                if header_str == 'EVTH':
                    shower_count += 1
                    current_shower = shower_count
                    # print(f"Начало ливня {current_shower} (найден EVTH)")
                elif header_str == 'EVTE':
                    if current_shower is not None:
                        idx_ne = 175 + 10 - 1
                        idx_s  = 185 + 10 - 1  
                        if len(subblock) > max(idx_ne, idx_s):
                            ne_nkg = subblock[idx_ne]
                            s_nkg  = subblock[idx_s]
                            print(f"Ливень {current_shower}: NeNKG = {ne_nkg}, sNKG = {s_nkg}")
                            params_dict[current_shower-1] = [ne_nkg, s_nkg]
                            current_shower = None
                        else:
                            print(f"Ливень {current_shower}: подблок EVTE слишком короткий")
                elif header_str == 'RUNE':
                    print("Найден блок RUNE, завершаем обработку.")
                    return

file_path = 'data/p30/pDAT163010' 
parse_corsika_showers(file_path)


Ливень 1: NeNKG = 391831.96875, sNKG = 1.3683958053588867
Ливень 2: NeNKG = 1135387.0, sNKG = 1.259588599205017
Ливень 3: NeNKG = 307665.75, sNKG = 1.3709015846252441
Ливень 4: NeNKG = 723334.4375, sNKG = 1.371324896812439
Ливень 5: NeNKG = 414350.84375, sNKG = 1.3645058870315552
Найден блок RUNE, завершаем обработку.


In [6]:
df = pd.DataFrame.from_dict(params_dict, orient='index', columns=['Ne', 's'])
df.index.name = 'ID'
df.reset_index(inplace=True)  

df.columns = ['event_id', 'Ne', 's']

theta, phi, E_prim = [], [], []
for i in df['event_id']:
    df_event = pd.read_csv(save_path + f'/event_{i}.csv')
    theta.append(df_event.iloc[0]['theta_prim'])
    phi.append(df_event.iloc[0]['phi_prim'])
    E_prim.append(df_event.iloc[0]['energy_prim'])
    
df['theta'] = theta
df['phi'] = phi
df['E_prim'] = E_prim
  

df.to_csv(save_path + '/_params.csv', index=False, encoding='utf-8')